# Descripción del proyecto
La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del sprint de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.#

Desarrolla un modelo con la mayor *exactitud* posible. En este proyecto, el umbral de *exactitud* es 0.75. Usa el dataset para comprobar la *exactitud*.

## Instrucciones del proyecto.

1. Abre y examina el archivo de datos. Dirección al archivo:/datasets/users_behavior.csv Descarga el dataset
2. Segmenta los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.
3. Investiga la calidad de diferentes modelos cambiando los hiperparámetros. Describe brevemente los hallazgos del estudio.
4. Comprueba la calidad del modelo usando el conjunto de prueba.
5. Tarea adicional: haz una prueba de cordura al modelo. Estos datos son más complejos que los que habías usado antes así que no será una tarea fácil. Más adelante lo veremos con más detalle.

## Descripción de datos
Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario. La información dada es la siguiente:

* сalls — número de llamadas,
* minutes — duración total de la llamada en minutos,
* messages — número de mensajes de texto,
* mb_used — Tráfico de Internet utilizado en MB,
* is_ultra — plan para el mes actual (Ultra - 1, Smart - 0).

## Evaluación del proyecto
Hemos definido los criterios de evaluación para el proyecto. Lee esto con atención antes de pasar al ejercicio. 

Esto es lo que los revisores buscarán cuando evalúen tu proyecto:

* ¿Cómo leíste los datos después de descargarlos?
* ¿Segmentaste correctamente los datos en conjuntos de entrenamiento, validación y prueba?
* ¿Cómo escogiste el tamaño de los conjuntos?
* ¿Evaluaste correctamente la calidad del modelo?
* ¿Qué modelos e hiperparámentros usaste?
* ¿Cuáles fueron tus hallazgos?
* ¿Probaste los modelos correctamente?
* ¿Cuál es tu puntuación de exactitud?
* ¿Te ceñiste a la estructura del proyecto y mantuviste limpio el código?*

## Análisis Exploratorio

In [39]:
# Importo mis librerias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [40]:
# Importo y exploro mi dataframe
df = pd.read_csv('datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [41]:
# Usemos info para ver si los datos son del tipo correcto
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [42]:
# Busco valores duplicados
print(f'El dataframe tiene una cantidad de {df.duplicated().sum()} valores duplicados')

El dataframe tiene una cantidad de 0 valores duplicados


Mi *df* no tiene valores duplicados, y todos los datos se ven del tipo correcto, quiza podria pasar el numero de mensajes a **INT** pero en este caso lo dejare así por si necesito tener alguna fraccion, o llego a ocupar algun promedio del mismo.

In [43]:
# Inicializo mis variables
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

En este caso, aplicaré una división del 60%, 20% y 20% para los conjuntos de entrenamiento, validación y prueba, respectivamente.  
Como no se cuenta con un segundo conjunto de datos para validación, realizaré la partición directamente a partir del dataset original utilizando la función train_test_split de Scikit-learn.

In [44]:
# Split train/validation/test

# Separo para obtener 60% de mis valores para train, el 40% restante lo nombro both para aplicarle un segundo split
features_train, features_both, target_train, target_both = train_test_split(features, target, test_size=0.4, random_state=54321)

# Uso mi segundo split para obtener un 20% y 20% de mi valor total, aqui aplico un .5 de test size para dividirlo correctamente
features_valid, features_test, target_valid, target_test = train_test_split(features_both, target_both, test_size=0.4, random_state=54321)

In [45]:
# Modelo DecisionTree Classifier

# Inicializo mis valores en 0
best_score = 0
best_depth = None
best_model_tree = None

# Itero sobre depth [1:11] para definir la mejor profundidad
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=54321, max_depth=depth)
    model.fit(features_train, target_train)
    score = model.score(features_valid, target_valid)
    
    # Agrego un condicionamiento para guardar la opcion con accuracy mas alto
    if score > best_score:
        best_score = score
        best_depth = depth
        best_model_tree = model

print(f"Mejor exactitud: {best_score:.4f} con max_depth = {best_depth}")

Mejor exactitud: 0.7847 con max_depth = 10


In [53]:
# Modelo de Bosque Aleatorio

# Inicializo mis valores en 0
best_score = 0
best_est = 0
best_model_forest = None

# Inicio un ciclo for donde veo de 10 arbol a 51 en multiplos de 10
for est in range(10, 51, 10): 
    # Inicio un segundo ciclo for para ver la mejor profundiad de arbol de 1 a 10
    for depth in range(1, 11):  # profundidad
        model = RandomForestClassifier(random_state=54321, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        score = model.score(features_valid, target_valid)
        if score > best_score:
            best_score = score
            best_est = est
            best_depth = depth
            best_model_forest = model

print(f"Mejor Forest es {best_est} arboles, con profundidad de {best_depth}, con una exactitud de: {best_score}")

Mejor Forest es 10 arboles, con profundidad de 10, con una exactitud de: 0.8054474708171206


In [54]:
# Modelo de Regresion Logistica

model = LogisticRegression(random_state=54321, solver='liblinear')
model.fit(features_train, target_train)
score = model.score(features_valid, target_valid)

print('Accuracy del modelo de regresión logística en el conjunto de entrenamiento:', score)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.6964980544747081


El accuracy mas grande en el dataset de entrenamiento me lo dio el modelo de RandomForest con hiperparametros de 10 arboles y profundidad de 10 y un accuracy de 80.54%. Seguido de mi DecisionTree con hiperparametro de profundidad de 10 y un accuracy del 78.47%, finalmente fue mi modelo de regresión lineal con un accuracy de 69.64%.

In [57]:
# Entreno mi mejor modelo con los mejores parametros para evaluarlo en mi conjunto de validación y de prueba
print('El mejor modelo fue:', best_model_forest)
best_model_forest.fit(features_train, target_train)

# Accuracy en mi set de Validación
valid_predictions = best_model_forest.predict(features_valid)
valid_accuracy = accuracy_score(target_valid, valid_predictions)
print('\nAccuracy en mi set de validación es de:', valid_accuracy)

# Accuracy en mi set de Prueba
test_predictions = best_model_forest.predict(features_test)
test_accuracy = accuracy_score(target_test, test_predictions)
print('\nAccuracy en mi set de test es de:', test_accuracy)

El mejor modelo fue: RandomForestClassifier(max_depth=10, n_estimators=10, random_state=54321)

Accuracy en mi set de validación es de: 0.8054474708171206

Accuracy en mi set de test es de: 0.8252427184466019


Para este proyecto se tomo el dataset y se entrenaron modelos de arbol, bosque y regresion para ver cual me daba un resultado mas exacto. En este caso el mejor modelo fuer RandomForest con una porofundidad de 10 y 10 arboles, un accuracy del 82.52% en mi dataset de prueba. Superando el 75% que se pedia. Definitivamente un modelo que puede predecir muy bien.